<a href="https://colab.research.google.com/github/wenjunsun/Covid-19-analysis-with-uw-ubicomp/blob/master/data_and_pre-processing/include_state_with_no_stay_at_home_order.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We found out that our current dataset ignores the counties/states where they didn't have a stay at home order at all. Data somehow got deleted when we either merge them together or some other operations. Let's find out what the cause is and create a new dataset that includes the counties withuot shelter in place order. \\
This is because we want to do causal inference and see if shelter in place order does change people's behavior.

In [ ]:
cd drive/My\ Drive/COVID\ 19\ data\ analysis

/content/drive/My Drive/COVID 19 data analysis


In [ ]:
ls

 aggreagate_data_on_county.ipynb
 agg_social_dist
 agg_social_dist_2.csv
 compare_30_days_before_and_after.ipynb
'Copy of COVID-19 US state policy database.csv'
 days_since.csv
 graphs.ipynb
 include_state_with_no_stay_at_home_order.ipynb
 merge_mobility_with_first_case_and_shelter_in_place.ipynb
 simple_analysis_graph.ipynb
 small_data.csv
 social_dist_all_trimmed.csv
 social_dist_with_dates
 social_dist_with_days_since


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('social_dist_with_days_since')

In [ ]:
data.shape

(27850773, 22)

so merge days since covid and social_dist data is correct, since we have the correct number of rows.

In [ ]:
agg_data = pd.read_csv('agg_social_dist')

In [ ]:
agg_data.isnull().sum()

Unnamed: 0                              0
date_range_start                        0
date_range_end                          0
state                                   0
state_code                              0
cnamelong                               0
county_code                             0
days_since_first_case                   0
days_since_shelter                      0
candidate_device_count                  0
device_count                            0
completely_home_device_count            0
part_time_work_behavior_devices         0
full_time_work_behavior_devices         0
delivery_behavior_devices               0
median_home_dwell_time                  0
median_non_home_dwell_time         301341
median_percentage_time_home        301341
distance_traveled_from_home           727
dtype: int64

We see that our aggregate data has 0 null values in days_since_first_case, which shouldn't be the case. 

In [ ]:
days_since = pd.read_csv("days_since.csv")

In [ ]:
days_since[days_since['Date - shelter in place'].isnull()]

,Unnamed: 0,Date - first case,Date - first death,Date - reopening,Date - shelter in place,Date - shelter in place ends,cnamelong,county,county_code,state,state_code,state_name
111,111,2020-03-30,NaN,NaN,NaN,NaN,Arkansas County,1.0,5001.0,5.0,AR,Arkansas
112,112,2020-03-29,NaN,NaN,NaN,NaN,Ashley County,3.0,5003.0,5.0,AR,Arkansas
113,113,2020-03-28,NaN,NaN,NaN,NaN,Baxter County,5.0,5005.0,5.0,AR,Arkansas
114,114,2020-03-22,NaN,NaN,NaN,NaN,Benton County,7.0,5007.0,5.0,AR,Arkansas
115,115,2020-03-22,NaN,NaN,NaN,NaN,Boone County,9.0,5009.0,5.0,AR,Arkansas
...,...,...,...,...,...,...,...,...,...,...,...,...
3151,3151,2020-03-31,2020-04-01,NaN,NaN,NaN,Saipan Municipality,110.0,69110.0,69.0,MP,Northern Mariana Islands
3152,3152,2020-03-31,2020-04-01,NaN,NaN,NaN,Tinian Municipality,120.0,69120.0,69.0,MP,Northern Mariana Islands
3231,3231,2020-03-16,2020-04-05,NaN,NaN,NaN,St. Croix Island,10.0,78010.0,78.0,VI,Virgin Islands
3232,3232,2020-03-16,2020-04-05,NaN,NaN,NaN,St. John Island,20.0,78020.0,78.0,VI,Virgin Islands


In [ ]:
days_since[days_since['Date - first case'].isnull()]

,Unnamed: 0,Date - first case,Date - first death,Date - reopening,Date - shelter in place,Date - shelter in place ends,cnamelong,county,county_code,state,state_code,state_name
67,67,NaN,NaN,2020-04-24,2020-03-28 00:00:00,2020-04-24,Aleutians East Borough,13.0,2013.0,2.0,AK,Alaska
68,68,NaN,NaN,2020-04-24,2020-03-28 00:00:00,2020-04-24,Aleutians West Census Area,16.0,2016.0,2.0,AK,Alaska
71,71,NaN,NaN,2020-04-24,2020-03-28 00:00:00,2020-04-24,Bristol Bay Borough,60.0,2060.0,2.0,AK,Alaska
72,72,NaN,NaN,2020-04-24,2020-03-28 00:00:00,2020-04-24,Denali Borough,68.0,2068.0,2.0,AK,Alaska
73,73,NaN,NaN,2020-04-24,2020-03-28 00:00:00,2020-04-24,Dillingham Census Area,70.0,2070.0,2.0,AK,Alaska
...,...,...,...,...,...,...,...,...,...,...,...,...
3143,3143,NaN,NaN,NaN,NaN,NaN,Eastern District,10.0,60010.0,60.0,AS,American Samoa
3144,3144,NaN,NaN,NaN,NaN,NaN,Manu'a District,20.0,60020.0,60.0,AS,American Samoa
3145,3145,NaN,NaN,NaN,NaN,NaN,Rose Island,30.0,60030.0,60.0,AS,American Samoa
3146,3146,NaN,NaN,NaN,NaN,NaN,Swains Island,40.0,60040.0,60.0,AS,American Samoa


In [ ]:
agg_data[agg_data['cnamelong'] == 'Arkansas County']

,Unnamed: 0,date_range_start,date_range_end,state,state_code,cnamelong,county_code,days_since_first_case,days_since_shelter,candidate_device_count,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,delivery_behavior_devices,median_home_dwell_time,median_non_home_dwell_time,median_percentage_time_home,distance_traveled_from_home


In [ ]:
data[data['cnamelong'] == 'Arkansas County']

,Unnamed: 0,date_range_start,date_range_end,state,state_code,cnamelong,county_code,origin_census_block_group,candidate_device_count,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,delivery_behavior_devices,median_home_dwell_time,median_non_home_dwell_time,median_percentage_time_home,distance_traveled_from_home,Date - first case,Date - shelter in place,days_since_first_case,days_since_shelter
4462,4462,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,5.0,AR,Arkansas County,5001,50014808001,NaN,50,13,3,10,NaN,867,NaN,NaN,13707.0,2020-03-30,NaN,-89.0,NaN
22338,22338,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,5.0,AR,Arkansas County,5001,50014803002,NaN,23,4,1,3,NaN,631,NaN,NaN,2215.0,2020-03-30,NaN,-89.0,NaN
23285,23285,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,5.0,AR,Arkansas County,5001,50014801001,NaN,50,11,1,3,NaN,900,NaN,NaN,21330.0,2020-03-30,NaN,-89.0,NaN
25150,25150,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,5.0,AR,Arkansas County,5001,50014803001,NaN,165,35,10,33,NaN,761,NaN,NaN,4171.0,2020-03-30,NaN,-89.0,NaN
30503,30503,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,5.0,AR,Arkansas County,5001,50014804001,NaN,40,14,5,5,NaN,510,NaN,NaN,8212.0,2020-03-30,NaN,-89.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27796641,27796641,2020-05-06T00:00:00-05:00,2020-05-07T00:00:00-05:00,5.0,AR,Arkansas County,5001,50014801002,102.0,61,6,9,2,1.0,439,279.0,61.0,16650.0,2020-03-30,NaN,37.0,NaN
27804024,27804024,2020-05-06T00:00:00-05:00,2020-05-07T00:00:00-05:00,5.0,AR,Arkansas County,5001,50014804003,50.0,19,7,1,1,1.0,787,28.0,88.0,1552.0,2020-03-30,NaN,37.0,NaN
27831667,27831667,2020-05-06T00:00:00-05:00,2020-05-07T00:00:00-05:00,5.0,AR,Arkansas County,5001,50014805001,188.0,119,19,21,2,2.0,824,204.0,78.0,2127.0,2020-03-30,NaN,37.0,NaN
27834833,27834833,2020-05-06T00:00:00-05:00,2020-05-07T00:00:00-05:00,5.0,AR,Arkansas County,5001,50014808002,86.0,45,10,6,2,1.0,520,201.0,69.0,10587.0,2020-03-30,NaN,37.0,NaN


Did a little digging, found out that groupby function ignores the NaN values in its groupby columns. This can be an issue since we want to preserve NaN days_since_shelter. We need to update our pandas package to 1.1 to do this.

In [ ]:
!pip install --upgrade --pre pandas==1.1.0rc0

     |████████████████████████████████| 10.5MB 3.1MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=1.0.0; python_version >= "3.0", but you'll have pandas 1.1.0rc0 which is incompatible.
  Found existing installation: pandas 1.0.5
    Uninstalling pandas-1.0.5:
      Successfully uninstalled pandas-1.0.5


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
wm = lambda x: np.average(x, weights= data.loc[x.index, "device_count"])

In [ ]:
agg_data = data.groupby(["date_range_start",'date_range_end','state','state_code','cnamelong','county_code','days_since_first_case','days_since_shelter'], dropna=False).agg(
    candidate_device_count = ('candidate_device_count', 'sum'),
    device_count = ('device_count', 'sum'),
    completely_home_device_count = ('completely_home_device_count', 'sum'),
    part_time_work_behavior_devices = ('part_time_work_behavior_devices', 'sum'),
    full_time_work_behavior_devices = ('full_time_work_behavior_devices', 'sum'),
    delivery_behavior_devices = ('delivery_behavior_devices', 'sum'),
    median_home_dwell_time = ('median_home_dwell_time',wm),
    median_non_home_dwell_time = ('median_non_home_dwell_time', wm),
    median_percentage_time_home = ('median_percentage_time_home', wm), 
    distance_traveled_from_home = ('distance_traveled_from_home', wm)).reset_index()

In [ ]:
agg_data.shape

(414853, 18)

check that our agg_data has the counties that has NaN on days_since_shelter column. Arkansas County is one of the county that never implemented shelter in place, let's check we have rows about that county.

In [ ]:
agg_data[agg_data['cnamelong'] == 'Arkansas County']

,date_range_start,date_range_end,state,state_code,cnamelong,county_code,days_since_first_case,days_since_shelter,candidate_device_count,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,delivery_behavior_devices,median_home_dwell_time,median_non_home_dwell_time,median_percentage_time_home,distance_traveled_from_home
1319,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,5.0,AR,Arkansas County,5001,-89.0,NaN,0.0,1343,349,69,208,0.0,773.294118,NaN,NaN,7323.903202
4584,2020-01-02T00:00:00-06:00,2020-01-03T00:00:00-06:00,5.0,AR,Arkansas County,5001,-88.0,NaN,0.0,1360,313,182,291,0.0,723.328676,NaN,NaN,7974.714706
7852,2020-01-03T00:00:00-06:00,2020-01-04T00:00:00-06:00,5.0,AR,Arkansas County,5001,-87.0,NaN,0.0,1352,263,209,299,0.0,705.217456,NaN,NaN,7068.951923
11120,2020-01-04T00:00:00-06:00,2020-01-05T00:00:00-06:00,5.0,AR,Arkansas County,5001,-86.0,NaN,0.0,1330,278,94,241,0.0,740.069173,NaN,NaN,8935.468421
14387,2020-01-05T00:00:00-06:00,2020-01-06T00:00:00-06:00,5.0,AR,Arkansas County,5001,-85.0,NaN,0.0,1321,315,108,172,0.0,745.117335,NaN,NaN,5317.486752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399938,2020-05-02T00:00:00-05:00,2020-05-03T00:00:00-05:00,5.0,AR,Arkansas County,5001,33.0,NaN,2756.0,1484,322,99,36,51.0,662.575472,122.676550,80.960916,6175.187332
403184,2020-05-03T00:00:00-05:00,2020-05-04T00:00:00-05:00,5.0,AR,Arkansas County,5001,34.0,NaN,2756.0,1432,389,70,27,40.0,760.643156,91.990922,87.836592,5338.799581
406430,2020-05-04T00:00:00-05:00,2020-05-05T00:00:00-05:00,5.0,AR,Arkansas County,5001,35.0,NaN,2756.0,1530,304,156,51,43.0,668.394118,158.271242,74.403922,6051.807843
409673,2020-05-05T00:00:00-05:00,2020-05-06T00:00:00-05:00,5.0,AR,Arkansas County,5001,36.0,NaN,2756.0,1536,366,202,71,54.0,551.048828,157.408203,71.987630,6421.222005


Great! Now we have the rows which has NaN in days_since_shelter column, because they never have shelter in place order from state/county! Let's save this to another csv file

In [ ]:
agg_data.to_csv("agg_social_dist_2.csv", index=False)

In [ ]:
data = pd.read_csv("agg_social_dist_2.csv")

In [ ]:
data

,date_range_start,date_range_end,state,state_code,cnamelong,county_code,days_since_first_case,days_since_shelter,candidate_device_count,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,delivery_behavior_devices,median_home_dwell_time,median_non_home_dwell_time,median_percentage_time_home,distance_traveled_from_home
0,2020-01-01T00:00:00+10:00,2020-01-02T00:00:00+10:00,66.0,GU,Guam,66010,-75.0,NaN,0.0,310,98,41,72,0.0,395.312903,NaN,NaN,3.722158e+06
1,2020-01-01T00:00:00+10:00,2020-01-02T00:00:00+10:00,69.0,MP,Rota Municipality,69100,-90.0,NaN,0.0,8,1,1,1,0.0,1069.000000,NaN,NaN,7.430000e+02
2,2020-01-01T00:00:00+10:00,2020-01-02T00:00:00+10:00,69.0,MP,Saipan Municipality,69110,-90.0,NaN,0.0,303,121,33,35,0.0,779.036304,NaN,NaN,NaN
3,2020-01-01T00:00:00+10:00,2020-01-02T00:00:00+10:00,69.0,MP,Tinian Municipality,69120,-90.0,NaN,0.0,12,1,1,3,0.0,0.000000,NaN,NaN,2.156500e+04
4,2020-01-01T00:00:00-04:00,2020-01-02T00:00:00-04:00,23.0,ME,Aroostook County,23003,-93.0,-91.0,0.0,56,20,5,7,0.0,477.000000,NaN,NaN,2.278000e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414848,2020-05-06T00:00:00-10:00,2020-05-07T00:00:00-10:00,15.0,HI,Hawaii County,15001,57.0,42.0,17698.0,7998,3075,490,141,153.0,719.768317,39.970743,92.613028,9.709844e+03
414849,2020-05-06T00:00:00-10:00,2020-05-07T00:00:00-10:00,15.0,HI,Honolulu County,15003,60.0,42.0,90915.0,42788,16900,2961,835,1155.0,840.354726,31.484038,94.495022,5.268723e+03
414850,2020-05-06T00:00:00-10:00,2020-05-07T00:00:00-10:00,15.0,HI,Kauai County,15007,53.0,42.0,6710.0,3181,1092,204,76,74.0,738.153725,52.559572,89.952216,5.898517e+03
414851,2020-05-06T00:00:00-10:00,2020-05-07T00:00:00-10:00,15.0,HI,Maui County,15009,52.0,42.0,16007.0,7895,3052,546,155,169.0,881.631412,38.644205,94.809626,5.629950e+03


In [ ]:
days_since = pd.read_csv("days_since.csv")

In [ ]:
days_since.head()

,Unnamed: 0,Date - first case,Date - first death,Date - reopening,Date - shelter in place,Date - shelter in place ends,cnamelong,county,county_code,state,state_code,state_name
0,0,2020-03-24,2020-04-07,2020-04-30,2020-04-04 00:00:00,2020-04-30,Autauga County,1.0,1001.0,1.0,AL,Alabama
1,1,2020-03-15,2020-03-29,2020-04-30,2020-04-04 00:00:00,2020-04-30,Baldwin County,3.0,1003.0,1.0,AL,Alabama
2,2,2020-04-03,2020-04-29,2020-04-30,2020-04-04 00:00:00,2020-04-30,Barbour County,5.0,1005.0,1.0,AL,Alabama
3,3,2020-03-30,2020-05-08,2020-04-30,2020-04-04 00:00:00,2020-04-30,Bibb County,7.0,1007.0,1.0,AL,Alabama
4,4,2020-03-25,2020-05-17,2020-04-30,2020-04-04 00:00:00,2020-04-30,Blount County,9.0,1009.0,1.0,AL,Alabama


In [ ]:
days_since = days_since[['county_code', 'Date - first case', 'Date - shelter in place']]

In [ ]:
merged_data = data.merge(days_since, how='left', on='county_code')

In [ ]:
merged_data.head()

,date_range_start,date_range_end,state,state_code,cnamelong,county_code,days_since_first_case,days_since_shelter,candidate_device_count,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,delivery_behavior_devices,median_home_dwell_time,median_non_home_dwell_time,median_percentage_time_home,distance_traveled_from_home,Date - first case,Date - shelter in place
0,2020-01-01T00:00:00+10:00,2020-01-02T00:00:00+10:00,66.0,GU,Guam,66010,-75.0,NaN,0.0,310,98,41,72,0.0,395.312903,NaN,NaN,3.722158e+06,2020-03-16,NaN
1,2020-01-01T00:00:00+10:00,2020-01-02T00:00:00+10:00,69.0,MP,Rota Municipality,69100,-90.0,NaN,0.0,8,1,1,1,0.0,1069.000000,NaN,NaN,7.430000e+02,2020-03-31,NaN
2,2020-01-01T00:00:00+10:00,2020-01-02T00:00:00+10:00,69.0,MP,Saipan Municipality,69110,-90.0,NaN,0.0,303,121,33,35,0.0,779.036304,NaN,NaN,NaN,2020-03-31,NaN
3,2020-01-01T00:00:00+10:00,2020-01-02T00:00:00+10:00,69.0,MP,Tinian Municipality,69120,-90.0,NaN,0.0,12,1,1,3,0.0,0.000000,NaN,NaN,2.156500e+04,2020-03-31,NaN
4,2020-01-01T00:00:00-04:00,2020-01-02T00:00:00-04:00,23.0,ME,Aroostook County,23003,-93.0,-91.0,0.0,56,20,5,7,0.0,477.000000,NaN,NaN,2.278000e+04,2020-04-03,2020-04-01 00:00:00


In [ ]:
merged_data.drop(['date_range_end'], axis=1, inplace=True)

In [ ]:
merged_data.to_csv("agg_social_dist_2.csv", index=False)

In [ ]:
data = pd.read_csv("agg_social_dist_2.csv")

In [ ]:
data.head()

,date_range_start,state,state_code,cnamelong,county_code,days_since_first_case,days_since_shelter,candidate_device_count,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,delivery_behavior_devices,median_home_dwell_time,median_non_home_dwell_time,median_percentage_time_home,distance_traveled_from_home,Date - first case,Date - shelter in place
0,2020-01-01T00:00:00+10:00,66.0,GU,Guam,66010,-75.0,NaN,0.0,310,98,41,72,0.0,395.312903,NaN,NaN,3.722158e+06,2020-03-16,NaN
1,2020-01-01T00:00:00+10:00,69.0,MP,Rota Municipality,69100,-90.0,NaN,0.0,8,1,1,1,0.0,1069.000000,NaN,NaN,7.430000e+02,2020-03-31,NaN
2,2020-01-01T00:00:00+10:00,69.0,MP,Saipan Municipality,69110,-90.0,NaN,0.0,303,121,33,35,0.0,779.036304,NaN,NaN,NaN,2020-03-31,NaN
3,2020-01-01T00:00:00+10:00,69.0,MP,Tinian Municipality,69120,-90.0,NaN,0.0,12,1,1,3,0.0,0.000000,NaN,NaN,2.156500e+04,2020-03-31,NaN
4,2020-01-01T00:00:00-04:00,23.0,ME,Aroostook County,23003,-93.0,-91.0,0.0,56,20,5,7,0.0,477.000000,NaN,NaN,2.278000e+04,2020-04-03,2020-04-01 00:00:00
